# Interactive parallel computing with ipyparallel

ipyparallel is a Python package and collection of scripts for controlling clusters for Jupyter.

ipyparallel contains the following scripts:

- ipcluster - start/stop a cluster
- ipcontroller - start a scheduler
- ipengine - start an engine


## Installing/activating ipyparallel

- first install the package: `conda install ipyparallel`
- to be able to launch parallel engines from the dashboard add `c.NotebookApp.server_extensions.append('ipyparallel.nbextension')` to the file `~/.jupyter/jupyter_notebook_config.py` (may need to create it)
- then enable the IPython Clusters tab in Jupyter by typing `ipcluster nbextension enable --user` in a terminal


### <font color="red"> *Exercise*: </font>
- Install and activate ipyparallel!

## How does it work?

- Launch IPython **engines** (Independent Python processes. Typically one per core)
- Create a `Client` that acts as a proxy to the engines
- Use client to launch tasks on the engines

### There are several ways to run parallel code

- **Direct interface** - access engines directly, explicitly through their identifiers
- **load-balanced interface** - submit job to scheduler which distributes to engines depending on load

### <font color="blue"> Demo, direct interface </blue>

In [17]:
from ipyparallel import Client

In [2]:
rc = Client()

The `ids` attribute of Client instance shows identifiers of engines that IPython detected

In [3]:
rc.ids

[0, 1]

### <font color="red"> *Exercise*: </font>
- what other attributes does rc have?


### <font color="blue"> Demo, %px magic </blue>

Use the %px (or %%px) magic to execute given python code in parallel

In [ ]:
%px?

In [7]:
%px import os, time

In [8]:
%px print(time.time())

[stdout:0] 1481120754.66
[stdout:1] 1481120754.66


In [10]:
%px a = os.getpid()

In [12]:
%px print(a)

[stdout:0] 62450
[stdout:1] 62453


In [15]:
%pxresult

[stdout:0] 62450
[stdout:1] 62453


Specify list of engines to run code on using `--targets`. Supports Python slicing

In [16]:
%%px --targets :-1
    print(os.getpid())

[stdout:0] 62450


### <font color="red"> *Exercise: Estimating $\pi$ with parallel Monte Carlo* </font>

In [28]:
import ipyparallel

In [29]:
clients = ipyparallel.Client()
clients.ids

[0, 1]

In [30]:
dview = clients[:]
#dview = clients.direct_view()
dview

<DirectView [0, 1]>

Let us first make a plot to illustrate what we want to do

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pylab as plt
import numpy as np

n = 5000
p = np.random.rand(n,2)
idx = np.sqrt(p[:,0]**2+p[:,1]**2) < 1
plt.plot(p[idx,0],p[idx,1],'b.') # point inside
plt.plot(p[idx==False,0],p[idx==False,1],'r.') # point outside
plt.axis([-0.1,1.1,-0.1,1.1]) 
plt.show()

### Task

 - take the following function mc_pi
 - run it in serial using 10$^6$ random points, and time it
 - run it in parallel using the map_sync method of clients
 - time it, and compare with serial execution

In [ ]:
import numpy as np
def mc_pi(n):
    count = 0
    p = np.random.rand(n,2)
    for i in range(n):
        x = p[i,0]
        y = p[i,1]
        if (x**2 + y**2) <= 1.0:
            count += 1
    return float(count)/float(n)